In [ ]:
!pip install waymo-open-dataset-tf-2-12-0==1.6.4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Cell 2
import os
# TODO: Enter the relative path in your Google Drive to the unzipped folder for hw1_code_submission.zip
FOLDERNAME = 'CS7643Group/Dataset/training' # e.g. 'cs7643/hw1/Code'

assert FOLDERNAME is not None, "[!] Enter the foldername."
working_directory = os.path.join("/content/drive/MyDrive/", FOLDERNAME)
assert os.path.exists(working_directory), "Make sure your FOLDERNAME is correct"
%cd $working_directory


In [ ]:
import glob

# Define the path to your training folder
tfrecord_folder = '/content/drive/MyDrive/CS7643Group/Dataset/training/'

# Get a list of all .tfrecord files in the folder
tfrecord_files = glob.glob(os.path.join(tfrecord_folder, '*.tfrecord*'))

# Check how many tfrecord files were found
print(f"Found {len(tfrecord_files)} .tfrecord files.")

In [ ]:
import math
import os
import uuid
import time

from matplotlib import cm
import matplotlib.animation as animation
import matplotlib.pyplot as plt

import numpy as np
from IPython.display import HTML
import itertools
import tensorflow as tf

from google.protobuf import text_format
from waymo_open_dataset.metrics.ops import py_metrics_ops
from waymo_open_dataset.metrics.python import config_util_py as config_util
from waymo_open_dataset.protos import motion_metrics_pb2

# If you use a custom conversion from Scenario to tf.Example, set the correct
# number of map samples here.
# num_map_samples = 30000

# # Example field definition
# roadgraph_features = {
#     'roadgraph_samples/dir': tf.io.FixedLenFeature(
#         [num_map_samples, 3], tf.float32, default_value=None
#     ),
#     'roadgraph_samples/id': tf.io.FixedLenFeature(
#         [num_map_samples, 1], tf.int64, default_value=None
#     ),
#     'roadgraph_samples/type': tf.io.FixedLenFeature(
#         [num_map_samples, 1], tf.int64, default_value=None
#     ),
#     'roadgraph_samples/valid': tf.io.FixedLenFeature(
#         [num_map_samples, 1], tf.int64, default_value=None
#     ),
#     'roadgraph_samples/xyz': tf.io.FixedLenFeature(
#         [num_map_samples, 3], tf.float32, default_value=None
#     ),
# }
# Features of other agents.
state_features = {
    'state/id':
        tf.io.FixedLenFeature([128], tf.float32, default_value=None),
    'state/type':
        tf.io.FixedLenFeature([128], tf.float32, default_value=None),
    'state/is_sdc':
        tf.io.FixedLenFeature([128], tf.int64, default_value=None),
    'state/tracks_to_predict':
        tf.io.FixedLenFeature([128], tf.int64, default_value=None),
    'state/current/bbox_yaw':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/height':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/length':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/timestamp_micros':
        tf.io.FixedLenFeature([128, 1], tf.int64, default_value=None),
    'state/current/valid':
        tf.io.FixedLenFeature([128, 1], tf.int64, default_value=None),
    'state/current/vel_yaw':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/velocity_x':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/velocity_y':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/width':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/x':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/y':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/z':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/future/bbox_yaw':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/height':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/length':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/timestamp_micros':
        tf.io.FixedLenFeature([128, 80], tf.int64, default_value=None),
    'state/future/valid':
        tf.io.FixedLenFeature([128, 80], tf.int64, default_value=None),
    'state/future/vel_yaw':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/velocity_x':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/velocity_y':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/width':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/x':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/y':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/z':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/past/bbox_yaw':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/height':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/length':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/timestamp_micros':
        tf.io.FixedLenFeature([128, 10], tf.int64, default_value=None),
    'state/past/valid':
        tf.io.FixedLenFeature([128, 10], tf.int64, default_value=None),
    'state/past/vel_yaw':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/velocity_x':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/velocity_y':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/width':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/x':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/y':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/z':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
}
print(len(state_features))
# traffic_light_features = {
#     'traffic_light_state/current/state':
#         tf.io.FixedLenFeature([1, 16], tf.int64, default_value=None),
#     'traffic_light_state/current/valid':
#         tf.io.FixedLenFeature([1, 16], tf.int64, default_value=None),
#     'traffic_light_state/current/x':
#         tf.io.FixedLenFeature([1, 16], tf.float32, default_value=None),
#     'traffic_light_state/current/y':
#         tf.io.FixedLenFeature([1, 16], tf.float32, default_value=None),
#     'traffic_light_state/current/z':
#         tf.io.FixedLenFeature([1, 16], tf.float32, default_value=None),
#     'traffic_light_state/past/state':
#         tf.io.FixedLenFeature([10, 16], tf.int64, default_value=None),
#     'traffic_light_state/past/valid':
#         tf.io.FixedLenFeature([10, 16], tf.int64, default_value=None),
#     'traffic_light_state/past/x':
#         tf.io.FixedLenFeature([10, 16], tf.float32, default_value=None),
#     'traffic_light_state/past/y':
#         tf.io.FixedLenFeature([10, 16], tf.float32, default_value=None),
#     'traffic_light_state/past/z':
#         tf.io.FixedLenFeature([10, 16], tf.float32, default_value=None),
# }

features_description = {}
# features_description.update(roadgraph_features)
features_description.update(state_features)
# features_description.update(traffic_light_features)

In [ ]:
import torch
FILENAME = '/content/drive/MyDrive/CS7643Group/Dataset/training/training_tfexample.tfrecord-00000-of-01000'


state_features_list = [
    'state/id',
    'state/type',
    'state/is_sdc',
    'state/tracks_to_predict',
    'state/current/bbox_yaw',
    'state/current/height',
    'state/current/length',
    'state/current/timestamp_micros',
    'state/current/valid',
    'state/current/vel_yaw',
    'state/current/velocity_x',
    'state/current/velocity_y',
    'state/current/width',
    'state/current/x',
    'state/current/y',
    'state/current/z',
    'state/future/bbox_yaw',
    'state/future/height',
    'state/future/length',
    'state/future/timestamp_micros',
    'state/future/valid',
    'state/future/vel_yaw',
    'state/future/velocity_x',
    'state/future/velocity_y',
    'state/future/width',
    'state/future/x',
    'state/future/y',
    'state/future/z',
    'state/past/bbox_yaw',
    'state/past/height',
    'state/past/length',
    'state/past/timestamp_micros',
    'state/past/valid',
    'state/past/vel_yaw',
    'state/past/velocity_x',
    'state/past/velocity_y',
    'state/past/width',
    'state/past/x',
    'state/past/y',
    'state/past/z'
]

dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')

j = 0
vehicles_list = []
vehicles = 0
state_id = []

for raw_record in dataset:
    j += 1
    if j == 200:
      break
    # Parse each raw record using the features description
    parsed_example = tf.io.parse_single_example(raw_record, features_description)
    # print(parsed_example['state/type'])
    numpy_array = parsed_example['state/type'].numpy()
    parsed_type = torch.from_numpy(numpy_array)
    count_ones = (parsed_type == 1.0).sum().int()
    vehicles += count_ones

    vehicle_indices = torch.where(parsed_type == 1.0)
    vehicle_indices_list = vehicle_indices[0].tolist()

    for i in vehicle_indices_list:
      # print(parsed_example['state/id'][i])
      data_point = []
      for key in state_features_list:
        data_point.append(parsed_example[key][i])

      vehicles_list.append(data_point)


    # print(parsed_example['state/id'][0])
    # print(vehicles_list[0]['state/id'])
    # print(parsed_example['state/id'][0])
    # print(vehicles_list[0]['state/id'])


    # Do something with the parsed example
    print(len(vehicles_list))
# 472 scenarios each with 128 agents
# 1 -> vehicles
# 2 -> pedestrians
# 3 -> cyclists
# -1 -> unknown
# 26150 cars in total across all the scenarios
# we are not sure if these are the actual number of cars

# create a list of all the vehicles
# make sure to take into account the time stamps





In [ ]:
import pickle

pickled_file_path = '/content/drive/MyDrive/CS7643Group/Dataset/picked_waymo.pkl'
with open(pickled_file_path, 'wb') as file:
  pickle.dump(vehicles_list, file)